# Amazon Bedrock AgentCore Policy - 세분화된 접근 제어 튜토리얼

## 개요

이 노트북은 **Amazon Bedrock AgentCore Policy**를 사용하여 AI 에이전트의 도구 호출에 대한 세분화된 접근 제어를 구현하는 방법을 설명합니다.

### 학습 내용

- [Amazon Cognito](docs/cognito.md)에서 [JWT 토큰](docs/jwt-authorizer.md)에 커스텀 클레임 추가 방법
- [Cedar 정책](docs/cedar-policy.md)으로 JWT 클레임 기반 접근 제어 구현
- 속성 기반 접근 제어(ABAC) 패턴 적용
- 다양한 시나리오에서 정책 테스트 및 검증

### 핵심 개념

| 개념 | 설명 | 문서 |
|------|------|------|
| **Policy Engine** | Cedar 정책을 저장하고 평가하는 엔진 | [Cedar Policy](docs/cedar-policy.md) |
| **Cedar Policy** | 누가(principal) 무엇을(action) 할 수 있는지 정의 | [Cedar Policy](docs/cedar-policy.md) |
| **Principal Tags** | JWT 클레임이 Cedar에서 접근되는 방식 | [JWT Authorizer](docs/jwt-authorizer.md) |
| **Gateway Authorizer** | JWT 토큰을 검증하는 컴포넌트 | [JWT Authorizer](docs/jwt-authorizer.md) |
| **Cognito** | 사용자 인증 및 토큰 발급 서비스 | [Amazon Cognito](docs/cognito.md) |

### 아키텍처

```
                                ┌───────────────────────┐
                                │  Policy Engine        │
                                │  (Cedar 정책)         │
                                └───────────┬───────────┘
                                            │ 연결됨
                                            ▼
┌─────────────────┐             ┌───────────────────────┐             ┌─────────────┐
│   Amazon        │  JWT 토큰   │  AgentCore Gateway    │             │   Lambda    │
│   Cognito       │────────────>│                       │────────────>│   Target    │
│   + Lambda      │  + 클레임   │                       │  허용 시    │   (도구)    │
└─────────────────┘             └───────────────────────┘             └─────────────┘
```

> 📚 **문서 참조**: 문제 발생 시 [문제 해결 가이드](docs/troubleshooting.md)를 확인하세요.

---

## Part 1: 환경 설정

필요한 라이브러리를 가져오고 설정을 초기화합니다.

### 환경 설정 옵션

**옵션 1: UV 가상환경 사용 (권장)**
```bash
cd ../00_setup
chmod +x create_uv_virtual_env.sh
./create_uv_virtual_env.sh AgentCorePolicy
```
생성된 커널을 Jupyter에서 선택하세요.

In [ ]:
import json
import sys
import time
from pathlib import Path

import boto3

# Add parent directory to path for common imports
sys.path.insert(0, str(Path.cwd().parent))

# common 폴더에서 유틸리티 함수 가져오기
from common.auth_utils import (
    get_bearer_token,
    decode_token,
    make_gateway_request,
    analyze_response,
    display_test_result,
)
from common.gateway_utils import (
    validate_and_fix_gateway_authorizer,
    attach_policy_engine_to_gateway,
)
from common.policy_utils import (
    get_policy_engine,
    create_cedar_policy,
    wait_for_policy_active,
    delete_policy,
    cleanup_existing_policies,
    ensure_policy_engine,
)
from common.cognito_utils import (
    create_lambda_function,
    configure_cognito_trigger,
)

print("✓ 라이브러리 및 유틸리티 함수 로드 완료")

### Step 1.1: Gateway 및 Cognito 설정

`gateway_config.json` 파일이 없으면 아래 셀을 실행하여 Gateway와 Cognito를 설정합니다.

> ⚠️ 이미 설정이 완료된 경우 이 셀을 건너뛰세요.

#### 생성되는 AWS 리소스

| 리소스 | 코드에서 정의한 이름 | 실제 생성되는 이름 (예시) | 설명 |
|--------|---------------------|--------------------------|------|
| Lambda Function | `RefundLambda` | `RefundLambda` | 환불 처리 샘플 함수 (Node.js 20.x) |
| IAM Role | `RefundLambda-execution-role` | `RefundLambda-execution-role` | Lambda 실행 권한 |
| AgentCore Gateway | `TestGWforPolicyEngine` | `testgwforpolicyengine-****` | MCP 프로토콜 엔드포인트 |
| Gateway Target | `RefundToolTarget` | `RefundToolTarget` | Gateway에 연결된 Lambda 도구 |
| [Cognito User Pool](docs/cognito.md) | (자동 생성) | Name: `agentcore-gateway-****`, ID: `us-east-1_****` | 인증 컨테이너, 모든 Cognito 리소스의 부모 |
| [Cognito Domain](docs/cognito.md#user-pool-domain-사용자-풀-도메인) | (자동 생성) | `agentcore-****.auth.{region}.amazoncognito.com` | OAuth2 토큰 엔드포인트 URL 제공 |
| [Cognito App Client](docs/cognito.md#app-client-앱-클라이언트) | (자동 생성) | `****` (M2M 클라이언트) | `client_id`/`client_secret`으로 토큰 요청 |
| [Cognito Resource Server](docs/cognito.md#resource-server-리소스-서버) | `TestGateway` | `TestGateway` | OAuth2 scope 정의 (`TestGateway/invoke`) |

> 📝 **참고**: Gateway ID와 Cognito 리소스는 자동 생성된 suffix/ID를 포함합니다.

In [ ]:
# Gateway 및 Cognito 설정 (gateway_config.json이 없는 경우에만 실행)
config_file = Path.cwd() / "gateway_config.json"

if not config_file.exists():
    print("gateway_config.json이 없습니다. Gateway 설정을 시작합니다...")
    %run setup-gateway.py --region us-east-1
else:
    print(f"✓ gateway_config.json이 이미 존재합니다: {config_file}")

### 생성된 오브젝트
#### (1) AgentCore Gateway
- ![Gateway.png](img/Gateway.png)
#### (2) Lambda Function
- ![refund_lambda.png](img/refund_lambda.png)
#### (3) Cognito User Pool
- ![cognito_user_pool](img/cognito_user_pool.png)
#### (4) Cognito Client
- ![Cognito_App_Clients.png](imgCognito_App_Clients.png)
#### (5) Cognito Domain
- ![Cognito_Domain.png](img/Cognito_Domain.png)
#### (6) Cognito Resource Server
- ![Cognito_Resource_Server.png](img/Cognito_Resource_Server.png)

### Step 1.2: 설정 파일 로드

`gateway_config.json`에서 Gateway 및 Cognito 설정을 로드합니다.

In [ ]:
# 설정 파일 로드
config_path = Path.cwd() / "gateway_config.json"

with open(config_path, "r") as f:
    CONFIG = json.load(f)

# 주요 값 추출
REGION = CONFIG["region"]
GATEWAY_URL = CONFIG["gateway_url"]
GATEWAY_ID = CONFIG["gateway_id"]
GATEWAY_ARN = CONFIG["gateway_arn"]
USER_POOL_ID = CONFIG["client_info"]["user_pool_id"]
CLIENT_ID = CONFIG["client_info"]["client_id"]
CLIENT_SECRET = CONFIG["client_info"]["client_secret"]
TOKEN_ENDPOINT = CONFIG["client_info"]["token_endpoint"]
POLICY_ENGINE_ID = CONFIG.get("policy_engine_id")

# 주요 변수 출력
print("✓ 설정 로드 완료")
print(f"  REGION: {REGION}")
print(f"  GATEWAY_ID: {GATEWAY_ID}")
print(f"  GATEWAY_URL: {GATEWAY_URL}")
print(f"  GATEWAY_ARN: {GATEWAY_ARN}")
print(f"  USER_POOL_ID: {USER_POOL_ID}")
print(f"  CLIENT_ID: {CLIENT_ID}")
print(f"  CLIENT_SECRET: {'*' * 10}...{CLIENT_SECRET[-4:] if len(CLIENT_SECRET) > 4 else '****'}")
print(f"  TOKEN_ENDPOINT: {TOKEN_ENDPOINT}")
print(f"  POLICY_ENGINE_ID: {POLICY_ENGINE_ID or '설정 안 됨'}")

#### AWS 클라이언트 초기화

In [ ]:
session = boto3.Session(region_name=REGION)

lambda_client = session.client("lambda")
cognito_client = session.client("cognito-idp")
iam_client = session.client("iam")
sts_client = session.client("sts")

# AgentCore Control 클라이언트 (Policy Engine 및 Gateway 관리용)
policy_client = session.client("bedrock-agentcore-control", region_name=REGION)
gateway_control_client = session.client("bedrock-agentcore-control", region_name=REGION)

# 계정 정보 조회
ACCOUNT_ID = sts_client.get_caller_identity()["Account"]

print("✓ AWS 클라이언트 초기화 완료")
print(f"  계정 ID: {ACCOUNT_ID}")
print(f"  리전: {REGION}")

### Step 1.3: [Gateway Authorizer](docs/jwt-authorizer.md) 설정 검증

> 📖 Gateway JWT Authorizer 설정에 대한 자세한 내용은 [JWT Authorizer 가이드](docs/jwt-authorizer.md#gateway-jwt-authorizer-설정)를 참조하세요.

In [ ]:
# Gateway Authorizer 설정 검증 및 필요시 수정
scope = CONFIG.get("client_info", {}).get("scope", "")
print("scope: ", scope)

validate_and_fix_gateway_authorizer(
    gateway_control_client=gateway_control_client,
    gateway_id=GATEWAY_ID,
    region=REGION,
    user_pool_id=USER_POOL_ID,
    client_id=CLIENT_ID,
    scope=scope
)

### Step 1.4: Policy Engine 확인 및 Gateway 연결

In [ ]:
# Policy Engine 확인 (없으면 생성)
POLICY_ENGINE_ID = ensure_policy_engine(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    create_if_missing=True
)

if not POLICY_ENGINE_ID:
    raise RuntimeError("Policy Engine을 생성하거나 찾을 수 없습니다")

# 설정 파일 업데이트
CONFIG["policy_engine_id"] = POLICY_ENGINE_ID
with open(config_path, "w") as f:
    json.dump(CONFIG, f, indent=2)
print(f"✓ 설정 파일에 policy_engine_id 저장됨")

In [ ]:
# Policy Engine을 Gateway에 연결
engine = get_policy_engine(policy_client, POLICY_ENGINE_ID)
policy_engine_arn = engine.get("policyEngineArn")

attach_policy_engine_to_gateway(
    gateway_control_client=gateway_control_client,
    gateway_id=GATEWAY_ID,
    policy_engine_arn=policy_engine_arn,
    mode="ENFORCE"
)

#### Policy Engine
생성된 Policy Engine 및 Gateway 에 연결됨.
![policy_engine.png](img/policy_engine.png)

---

## Part 2: 테스트 시나리오 1 - 부서 기반 접근 제어

**finance** 부서 사용자만 도구를 사용할 수 있도록 정책을 생성합니다.

### [Cedar 정책](docs/cedar-policy.md) 패턴

> `principal.hasTag()`와 `principal.getTag()`는 [JWT 토큰](docs/jwt-authorizer.md)의 클레임에 접근하는 방법입니다.

```cedar
permit(principal, action, resource)
when {
    principal.hasTag("department_name") &&
    principal.getTag("department_name") == "finance"
};
```

### Step 2.0: 기존 정책 정리

In [ ]:
# 테스트 전 기존 정책 정리
cleanup_existing_policies(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    require_confirmation=False  # 자동 삭제
)

# 생성된 정책 추적용 리스트
CREATED_POLICIES = []

### Step 2.1: Finance 부서 클레임으로 Lambda 설정

[Amazon Cognito](docs/cognito.md)의 **Pre Token Generation Lambda Trigger (V3_0)**를 사용하여 토큰에 커스텀 클레임을 추가합니다.

> 📖 Lambda 트리거 버전에 대한 자세한 내용은 [Cognito 가이드](docs/cognito.md#lambda-triggers-lambda-트리거)를 참조하세요.

In [ ]:
print("=" * 70)
print("테스트 시나리오 1: 부서 기반 접근 제어")
print("=" * 70)

# department_name = "finance" 클레임으로 Lambda 설정
claims_finance = {
    "department_name": "finance",
    "employee_level": "senior",
    "cost_center": "CC-1001",
}

lambda_arn = create_lambda_function(
    lambda_client=lambda_client,
    iam_client=iam_client,
    claims=claims_finance,
    account_id=ACCOUNT_ID,
    user_pool_id=USER_POOL_ID
)

configure_cognito_trigger(
    cognito_client=cognito_client,
    lambda_client=lambda_client,
    user_pool_id=USER_POOL_ID,
    lambda_arn=lambda_arn,
    region=REGION,
    account_id=ACCOUNT_ID
)

print("\n✓ Lambda에 설정된 클레임:")
print(json.dumps(claims_finance, indent=2, ensure_ascii=False))

#### Pre Token Generation Lambda Trigger (V3_0) 생성
##### CogNitor 에 Trigger 할 수 있게 연결된 람다 함수
![Lambda_triggers.png](img/Lambda_triggers.png)
##### 람다함수 정의: cognito-custom-claims-us-east-1_ZsuzHgyyj
아래와 같이 정의
```
            'accessTokenGeneration': {
                'claimsToAddOrOverride': {
            "department_name": "finance",
            "employee_level": "senior",
            "cost_center": "CC-1001"
}
            },

```
![lambda_trigger_function.png](img/lambda_trigger_function.png)

### Step 2.2: [토큰](docs/jwt-authorizer.md#jwt란)에 커스텀 클레임이 포함되었는지 확인

> ⚠️ 커스텀 클레임이 토큰에 없으면 [문제 해결 가이드](docs/troubleshooting.md#3-커스텀-클레임이-토큰에-없음)를 확인하세요.

In [ ]:
token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)

print(f"  Token:  {token}")

claims = decode_token(token)

print("\n🔍 토큰 클레임 (주요 항목):")
print(f"  department_name: {claims.get('department_name', '없음')}")
print(f"  employee_level: {claims.get('employee_level', '없음')}")
print(f"  cost_center: {claims.get('cost_center', '없음')}")
print(f"  client_id: {claims.get('client_id', '없음')}")
print(f"  scope: {claims.get('scope', '없음')}")

if claims.get("department_name") == "finance":
    print("\n✓ 커스텀 클레임이 토큰에 포함되었습니다")
else:
    print("\n⚠️ 커스텀 클레임이 없습니다 - Lambda 트리거 설정을 확인하세요")

### Step 2.3: 부서 검증용 [Cedar 정책](docs/cedar-policy.md) 생성

> 📖 `hasTag()`와 `getTag()` 사용법은 [Cedar Policy 문법](docs/cedar-policy.md#principal-tags-jwt-클레임-접근)을 참조하세요.

In [ ]:
policy_name = f"dept_policy_{int(time.time())}"

cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.hasTag("department_name") &&
    principal.getTag("department_name") == "finance"
}};'''

policy_id = create_cedar_policy(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description="finance 부서만 접근 허용"
)

if policy_id:
    CREATED_POLICIES.append(policy_id)
    
    print("\n⏳ 정책이 ACTIVE 상태가 될 때까지 대기 중...")
    if wait_for_policy_active(policy_client, POLICY_ENGINE_ID, policy_id):
        print("✓ 정책이 ACTIVE 상태입니다. 테스트 준비 완료.")
    else:
        print("\n⚠️ 정책이 ACTIVE 상태가 되지 않았습니다.")
else:
    print("\n✗ 정책 생성 실패")

##### Created Policy
![created_policy.png](img/created_policy.png)


### Step 2.4: Finance 부서로 테스트 (예상: 허용)

In [ ]:
print("\n" + "=" * 70)
print("테스트 1.1: department_name='finance'로 요청")
print("=" * 70)

token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-dept-finance"}
)

print("\n요청: RefundToolTarget___refund(amount=500)")
print("\n응답:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "Finance 부서는 허용되어야 함")

### Step 2.5: Engineering 부서로 테스트 (예상: 거부)

In [ ]:
print("\n" + "=" * 70)
print("테스트 1.2: department_name='engineering'으로 요청")
print("=" * 70)

# 다른 부서로 Lambda 업데이트
claims_engineering = {
    "department_name": "engineering",
    "employee_level": "senior",
    "cost_center": "CC-2001",
}

lambda_arn = create_lambda_function(
    lambda_client=lambda_client,
    iam_client=iam_client,
    claims=claims_engineering,
    account_id=ACCOUNT_ID,
    user_pool_id=USER_POOL_ID
)
print("\n✓ Lambda가 department_name='engineering'으로 업데이트됨")

# Lambda 변경 전파 대기
print("\n⏳ Lambda 변경 전파 대기 중...")
time.sleep(5)

# 새 토큰으로 테스트
token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)
claims = decode_token(token)
print(f"\n토큰 department_name: {claims.get('department_name')}")

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-dept-engineering"}
)

print("\n요청: RefundToolTarget___refund(amount=500)")
print("\n응답:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("DENIED", outcome, "Engineering 부서는 거부되어야 함")

##### 람다함수 업데이트 (부서 : Engineering): cognito-custom-claims-us-east-1_ZsuzHgyyj
![lambda_trigger_function_engineer.png](img/lambda_trigger_function_engineer.png)


---

## Part 3: 테스트 시나리오 2 - 그룹 기반 접근 제어

**admins** 그룹에 속한 사용자만 도구를 사용할 수 있도록 정책을 생성합니다.

### [Cedar 정책](docs/cedar-policy.md) 패턴

그룹은 [토큰](docs/jwt-authorizer.md)에서 문자열로 직렬화되므로, `like` 연산자를 사용합니다:

> 📖 `like` 연산자의 패턴 매칭에 대한 자세한 내용은 [Cedar Policy 문법](docs/cedar-policy.md#패턴-매칭-like-연산자)을 참조하세요.

```cedar
permit(principal, action, resource)
when {
    principal.hasTag("groups") &&
    principal.getTag("groups") like "*admins*"
};
```

In [ ]:
# 이전 정책 정리
print("=" * 70)
print("테스트 시나리오 2: 그룹 기반 접근 제어")
print("=" * 70)

print("\n이전 정책 정리 중...")
for pid in CREATED_POLICIES:
    delete_policy(policy_client, POLICY_ENGINE_ID, pid)
CREATED_POLICIES.clear()

### Step 3.1: Groups 클레임으로 Lambda 설정

In [ ]:
# "admins"를 포함한 그룹으로 Lambda 설정
claims_with_admins = {
    "groups": ["admins", "developers", "team-alpha"],
    "department_name": "finance",
    "employee_level": "senior",
}

lambda_arn = create_lambda_function(
    lambda_client=lambda_client,
    iam_client=iam_client,
    claims=claims_with_admins,
    account_id=ACCOUNT_ID,
    user_pool_id=USER_POOL_ID
)

configure_cognito_trigger(
    cognito_client=cognito_client,
    lambda_client=lambda_client,
    user_pool_id=USER_POOL_ID,
    lambda_arn=lambda_arn,
    region=REGION,
    account_id=ACCOUNT_ID
)

print("\n✓ Lambda에 설정된 그룹:")
print(f"   groups: {claims_with_admins['groups']}")

### Step 3.2: 그룹 검증용 [Cedar 정책](docs/cedar-policy.md) 생성

> 📖 `like` 연산자의 와일드카드 패턴은 [Cedar Policy 문법](docs/cedar-policy.md#패턴-매칭-like-연산자)을 참조하세요.

In [ ]:
policy_name = f"groups_policy_{int(time.time())}"

cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.hasTag("groups") &&
    principal.getTag("groups") like "*admins*"
}};'''

policy_id = create_cedar_policy(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description="admins 그룹만 접근 허용"
)

if policy_id:
    CREATED_POLICIES.append(policy_id)
    
    print("\n⏳ 정책이 ACTIVE 상태가 될 때까지 대기 중...")
    if wait_for_policy_active(policy_client, POLICY_ENGINE_ID, policy_id):
        print("✓ 정책이 ACTIVE 상태입니다.")

### Step 3.3: Admins 그룹으로 테스트 (예상: 허용)

In [ ]:
print("\n" + "=" * 70)
print("테스트 2.1: groups=['admins', 'developers', 'team-alpha']로 요청")
print("=" * 70)

token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-groups-admins"}
)

print("\n응답:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "'admins' 그룹 사용자는 허용되어야 함")

### Step 3.4: Admins 없이 테스트 (예상: 거부)

In [ ]:
print("\n" + "=" * 70)
print("테스트 2.2: groups=['developers', 'team-alpha']로 요청 (admins 없음)")
print("=" * 70)

# admins 없이 Lambda 업데이트
claims_no_admins = {
    "groups": ["developers", "team-alpha"],
    "department_name": "finance",
    "employee_level": "senior",
}

lambda_arn = create_lambda_function(
    lambda_client=lambda_client,
    iam_client=iam_client,
    claims=claims_no_admins,
    account_id=ACCOUNT_ID,
    user_pool_id=USER_POOL_ID
)

print("\n⏳ Lambda 변경 전파 대기 중...")
time.sleep(5)

token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)
claims = decode_token(token)
print(f"\n토큰 groups: {claims.get('groups')}")

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-groups-no-admins"}
)

print("\n응답:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("DENIED", outcome, "'admins' 없는 사용자는 거부되어야 함")

---

## Part 4: 고급 패턴

### 복합 조건 결합

여러 조건을 결합하여 복잡한 접근 제어 시나리오를 구현할 수 있습니다.

> 📖 [JWT Scope와 Cedar Policy의 차이](docs/jwt-authorizer.md#jwt-scope-vs-cedar-policy)를 이해하면 언제 복합 조건이 필요한지 알 수 있습니다.

In [ ]:
# 예시: 복합 정책 (부서 AND 금액 제한)
print("\n고급 패턴: 복합 조건")
print("=" * 70)

combined_cedar = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.hasTag("department_name") &&
    principal.getTag("department_name") == "finance" &&
    context.input.amount <= 1000
}};'''

print("복합 조건 Cedar 정책:")
print("-" * 60)
print(combined_cedar)
print("-" * 60)
print("\n이 정책은 다음 조건을 모두 충족할 때 허용:")
print("  ✓ 사용자가 finance 부서")
print("  ✓ AND 환불 금액이 $1000 이하")

### 패턴 매칭 (`like` 연산자)

| 패턴 | 매칭 |
|------|------|
| `"*admin*"` | "admin" 포함 |
| `"admin*"` | "admin"으로 시작 |
| `"*admin"` | "admin"으로 끝남 |
| `"team-*"` | "team-"으로 시작 |

In [ ]:
# 예시: 팀 기반 접근 와일드카드
print("\n고급 패턴: 와일드카드를 사용한 팀 기반 접근")
print("=" * 70)

team_cedar = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.hasTag("groups") &&
    principal.getTag("groups") like "*team-finance*"
}};'''

print("패턴 매칭 Cedar 정책:")
print("-" * 60)
print(team_cedar)
print("-" * 60)
print("\n이 정책은 다음과 같은 그룹을 허용:")
print("  ✓ 매칭: ['team-finance', 'developers']")
print("  ✓ 매칭: ['admins', 'team-finance-leads']")

---

## Part 5: 정리

테스트 중 생성된 정책을 삭제합니다.

In [ ]:
print("=" * 70)
print("정리")
print("=" * 70)

print(f"\n{len(CREATED_POLICIES)}개 정책 삭제 중...")
for pid in CREATED_POLICIES:
    delete_policy(policy_client, POLICY_ENGINE_ID, pid)

CREATED_POLICIES.clear()
print("\n✓ 정리 완료")

---

## 결론

이 튜토리얼을 완료했습니다! 다음을 배웠습니다:

✅ [Amazon Cognito](docs/cognito.md) Lambda 트리거로 [JWT 토큰](docs/jwt-authorizer.md)에 커스텀 클레임 추가  
✅ [Cedar 정책](docs/cedar-policy.md)으로 JWT 클레임 기반 접근 제어 구현  
✅ 부서 기반 접근 제어  
✅ 그룹 기반 접근 제어 (패턴 매칭)  
✅ 복합 조건으로 복잡한 접근 제어 시나리오 구현  

### 핵심 [Cedar](docs/cedar-policy.md) 문법 패턴

| 클레임 유형 | Cedar 문법 | 예시 |
|------------|------------|------|
| 문자열 (정확한 일치) | `principal.getTag("claim") == "value"` | `department_name == "finance"` |
| 문자열/배열 (포함) | `principal.getTag("claim") like "*value*"` | `groups like "*admins*"` |
| 입력 검증 | `context.input.field <= value` | `context.input.amount <= 1000` |

> 📝 **참고**: JWT 배열 클레임은 문자열로 직렬화되므로 `like` 연산자로 포함 여부를 확인합니다.

### 다음 단계

1. **프로덕션 적용** - 실제 AgentCore 배포에 이 패턴 적용
2. **정책 커스터마이즈** - 요구사항에 맞게 정책 수정
3. **모니터링 추가** - CloudWatch로 정책 거부 알람 설정

---

## 📚 문서 참조

| 문서 | 내용 |
|------|------|
| [Amazon Cognito](docs/cognito.md) | Cognito 개념, User Pool, OAuth2 플로우 |
| [Cedar Policy 문법](docs/cedar-policy.md) | Cedar 정책 언어 문법 및 예제 |
| [JWT Authorizer](docs/jwt-authorizer.md) | JWT Authorizer, Scope, Principal Tags |
| [문제 해결](docs/troubleshooting.md) | 자주 발생하는 오류 및 해결 방법 |